## *To perform the Test:*

**Connection Setup:**
   - Connect a Supernova with firmware version 2.5.0 or higher to a BMM350 Bosch target via the I3C HV bus. The Supernova will act as an I3C controller while the BMM350 as an I3C target.
  
     Other I3C targets may be used, but you should check their static address and DAA allowed methods to properly test the device and understand the outcome of the evaluation.

**Expected Messages and Notifications:**
   - For each command sent to the I3C controller, the notebook should show the response returned by it.
   - Calling i3cGetTargetDeviceTable, you can verify that the Target Devices Table is updated after each DAA method if run successfully: i3cRSTDAA (cleans the table), i3cSETDASA, i3cSETAASA, i3cENTDAA and i3cBusInit.
   
**Testing Process:**
   - Run each cell in order.
   - Look for immediate feedback:
     - The response for each command directed to a Supernova.


### 1. Startup

#### 1.1 Import the python package

In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *

#### 1.2 Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [2]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 1.3 List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [3]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '59314AC593A1C05C8163B6759994476',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': -1}]

#### 1.4 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [4]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 1.5 Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.


In [5]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 1.6 Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [6]:
import json

# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")

        # If the command is GET TARGET DEVICE TABLE, convert to json.
        if supernova_message["name"] == "I3C GET TARGET DEVICE TABLE" or supernova_message["name"] == "I3C TRANSFER":
            responseJson = json.dumps(supernova_message, indent=4)
            print(responseJson)

        else:
            print(supernova_message)
    
    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")

        print(system_message)

In [7]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

### 2. I3C Protocol API

The I3C Protocol API methods are described below.

### 2.1 Configure the Supernova as an I3C Controller

In [8]:
request_result = supernova.i3cControllerInit(getId())

>> New message from SUPERNOVA:
{'id': 1, 'command': 1, 'name': 'I3C CONTROLLER INIT', 'result': 'I3C_CONTROLLER_INIT_SUCCESS'}


### 2.2 Set I3C bus voltage

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [9]:
request_result = supernova.setI3cBusVoltage(getId(), 3300)

>> New message from SUPERNOVA:
{'id': 2, 'command': 97, 'name': 'SET I3C BUS VOLTAGE', 'result': 0}


### 2.3 I3C bus initialization

i3cInitBus will execute a RSTDAA and then try to perform the DAA in the order specified by the MIPI Alliance: SETDASA, SETAASA and finally ENTDAA.
  - SETDASA will be executed if there is at least one target in the table argument for i3cInitBus that allows SETDASA.
  - SETAASA will be executed if there is at least one target in the table argument for i3cInitBus that allows SETAASA but doesn't allow SETDASA.
  - ENTDAA will be executed if the table is not full or if there is at least one target in the table argument for i3cInitBus that allows ENTDAA but doesn't allow SETDASA nor SETAASA.

#### 2.3.1 Using i3cInitBus with no target entries as argument

The CCCs involved are:
- RSTDAA
- ENTDAA since the Target Devices Table is empty when invoking i3cInitBus


In [10]:
# Get the Target Devices Table from the Supernova. It must be empty before the bus initialization.
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 3,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


In [11]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cInitBus(getId())

>> New message from SUPERNOVA:
{'id': 4, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'I3C_BUS_INIT_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [12]:
# Get the Target Devices Table from the Supernova
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 5,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 0,
            "dynamicAddress": 8,
            "bcr": {
                "value": [
                    "0b00100110",
                    38,
                    "0x26"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Supports optional advanced capabilities. Use GETCAPS CCC to deremine which ones.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
                    "ibiRequestCapable": "Capable

#### 2.3.2 Using i3cInitBus with a target entry as argument

The CCCs involved are:
- RSTDAA 
- SETDASA since there is at least one entry that supports SETDASA 
- ENTDAA since the Target Devices Table is empty when invoking i3cInitBus


In [13]:
'''
If the static address of a target is known, sent it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
'''
BMM350_STATIC_ADDRESS = 0x14

'''
Select the dynamic address to be assigned to the I3C Target if the DAA method is a SETDASA.
'''
BMM350_DYNAMIC_ADDRESS = 0x0C

'''
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.
'''
features =  TargetInterruptRequest.ACCEPT_INTERRUPT_REQUEST.value |  \
            ControllerRoleRequest.REJECT_CRR.value |   \
            IBiTimestamp.DISABLE_IBIT.value |   \
            AssignmentFromStaticAddress.I3C_TARGET_SUPPORTS_SETDASA.value |   \
            DAaWithENTDAA.ENABLE_ENTDAA.value |   \
            TargetType.I3C_DEVICE.value |   \
            PendingReadCapability.SUPPORT_IBI_READ_CAPABILITY.value |   \
            ValidPID.HAS_VALID_PID.value

'''
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
'''
table = {0: {'staticAddress': BMM350_STATIC_ADDRESS,
        'dynamicAddress': BMM350_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'maxIbiPayloadLength': 9,
        'i3cFeatures': features}}

'''
Initialize the bus using the predefined target devices table.
'''
request_result = supernova.i3cInitBus(getId(), table)

>> New message from SUPERNOVA:
{'id': 6, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'I3C_BUS_INIT_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [14]:
# Get the update target devices table.
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 7,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 20,
            "dynamicAddress": 12,
            "bcr": {
                "value": [
                    "0b00000000",
                    0,
                    "0x00"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "No data bytes follow the accepted IBI.",
                    "ibiRequestCapable": "Not capable.",
                    "maxDataSpeedLimitation": "No Limitation."
          

#### 2.3.3 Using SETDASA

In [15]:
I3C_BROADCAST_ADDRESS = 0x7E

# Reset the bus issuing the RSTDAA command and clear the target device table.
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.I3C_BUS, I3C_BROADCAST_ADDRESS)

>> New message from SUPERNOVA:
{'id': 8, 'command': 7, 'name': 'I3C CLEAR FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [16]:
# Get the update target devices table
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 9,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


In [17]:
BMM350_STATIC_ADDRESS = 0x14
BMM350_DYNAMIC_ADDRESS = 0x0A

# Request a SETDASA
request_result = supernova.i3cSETDASA(getId(), BMM350_STATIC_ADDRESS, BMM350_DYNAMIC_ADDRESS, I3cPushPullTransferRate.PUSH_PULL_6_25_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_3_125_MHZ)

>> New message from SUPERNOVA:
{
    "id": 10,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "invalid_addresses": []
}


In [18]:
# Get the update target devices table
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 11,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 20,
            "dynamicAddress": 10,
            "bcr": {
                "value": [
                    "0b00000000",
                    0,
                    "0x00"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "No data bytes follow the accepted IBI.",
                    "ibiRequestCapable": "Not capable.",
                    "maxDataSpeedLimitation": "No Limitation."
         

#### 2.3.4 Using SETAASA

In [19]:
I3C_BROADCAST_ADDRESS = 0x7E

# Reset the bus issuing the RSTDAA command and clear the target device table.
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.I3C_BUS, I3C_BROADCAST_ADDRESS)

>> New message from SUPERNOVA:
{'id': 12, 'command': 7, 'name': 'I3C CLEAR FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [20]:
# Get the update target devices table
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 13,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


In [21]:
BMM350_STATIC_ADDRESS = 0x14

# Request a SETAASA
request_result = supernova.i3cSETAASA(getId(), [BMM350_STATIC_ADDRESS], I3cPushPullTransferRate.PUSH_PULL_6_25_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_3_125_MHZ)

>> New message from SUPERNOVA:
{
    "id": 14,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "invalid_addresses": []
}


In [22]:
# Get the update target devices table
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 15,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 20,
            "dynamicAddress": 20,
            "bcr": {
                "value": [
                    "0b00000000",
                    0,
                    "0x00"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "No data bytes follow the accepted IBI.",
                    "ibiRequestCapable": "Not capable.",
                    "maxDataSpeedLimitation": "No Limitation."
         

#### 2.3.5 Using ENTDAA

In [23]:
I3C_BROADCAST_ADDRESS = 0x7E

# Reset the bus issuing the RSTDAA command and clear the target device table.
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.I3C_BUS, I3C_BROADCAST_ADDRESS)

>> New message from SUPERNOVA:
{'id': 16, 'command': 7, 'name': 'I3C CLEAR FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [24]:
# Get the update target devices table.
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 17,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


In [25]:
'''
If the static address of a target is known, sent it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
'''
BMM350_STATIC_ADDRESS = 0x14

'''
Select the dynamic address to be assigned to the I3C Target if the DAA method is a SETDASA.
'''
BMM350_DYNAMIC_ADDRESS = 0x0C

'''
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.
'''
features =  TargetInterruptRequest.ACCEPT_INTERRUPT_REQUEST.value |  \
            ControllerRoleRequest.REJECT_CRR.value |   \
            IBiTimestamp.DISABLE_IBIT.value |   \
            DAaWithENTDAA.ENABLE_ENTDAA.value |   \
            TargetType.I3C_DEVICE.value |   \
            PendingReadCapability.SUPPORT_IBI_READ_CAPABILITY.value |   \
            ValidPID.HAS_VALID_PID.value

'''
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
'''
table = {0: {'staticAddress': BMM350_STATIC_ADDRESS,
        'dynamicAddress': BMM350_DYNAMIC_ADDRESS,
        'bcr': 0x26,
        'dcr': 0x43,
        'pid': [0x07, 0x70, 0x10, 0x33, 0x00, 0x00],
        'maxIbiPayloadLength': 9,
        'i3cFeatures': features}
        }

# Initialize and scan the I3C bus.
request_result = supernova.i3cENTDAA(getId(), table)

>> New message from SUPERNOVA:
{
    "id": 18,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "invalid_addresses": []
}


In [26]:
# Get the update target devices table.
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 19,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 20,
            "dynamicAddress": 12,
            "bcr": {
                "value": [
                    "0b00100110",
                    38,
                    "0x26"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Supports optional advanced capabilities. Use GETCAPS CCC to deremine which ones.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
                    "ibiRequestCapable": "Capa

### 3. Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [27]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}